# Практикум 5. 
Побудова асоцiативних правил
за алгоритмами Apriori та FP-росту

In [4]:
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori,association_rules
from mlxtend.frequent_patterns import fpgrowth
warnings.filterwarnings('ignore')

df=pd.read_csv('Var_8_Market_Basket_Analysis_2.csv', header=None, delimiter='\n')
df.head()
df = df[0].str.get_dummies(sep=',')
df.head()

,Bread,Butter,Cheese,Coffee Powder,Ghee,Lassi,Milk,Panner,Sugar,Sweet,Tea Powder,Yougurt
0,0,1,1,1,1,1,0,0,0,0,0,1
1,0,0,0,1,1,0,0,0,0,0,0,0
2,0,1,1,0,0,1,0,0,0,0,1,0
3,1,1,1,1,0,0,0,1,0,0,1,0
4,0,1,1,1,0,0,0,0,1,1,0,1


Побудувати частi набори та асоцiативнi правила (АП), використовую-
чи алгоритм Apriori. Дослiдити множини АП залежно вiд параметрiв
побудови АП:
• мiнiмальної пiдтримки,
• мiнiмальної достовiрностi,
• максимальної потужностi частих наборiв.
Пiдiбрати значення параметрiв алгоритму Apriori, якi призводять до
значущих АП. АП Rk вважати значущим, якщо покращення (improvement
або лiфт (lift)): Improv(Rk) > 1.

Apriori это алгоритм поиска ассоциативных правил,
которые генерируются на основе всех частых предметных наборов,
обнаруженных в транзакционной базе данных, и удовлетворяют заданному уровню поддержки и достоверности.

Алгоритм позволяет сократить пространство поиска благодаря свойству антимонотонности множеств,
которое утверждает, что если предметный набор
Z не является частым, то добавление к нему некоторого нового предмета
A не делает его таковым. Иными словами, если
Z не является частым, то и
Z+A также не будет им.


In [6]:
def create_rules(df, supp=0.1, conf=0, max_len=None, algorithm=apriori):
    frequent_itemssets=algorithm(df, min_support=supp, use_colnames=True, max_len=max_len, verbose=0)
    rules=association_rules(frequent_itemssets, metric='lift')
    rules=rules[(rules['lift'] >= 1) & (rules['confidence'] >= conf)]
    return rules

In [7]:
#попытаемся подобрать максимальное количество полезных правил
rules_stats=[]
for sup in range(1,10):
    for conf in range(1,10):
        for max_len in range(2,12):
            try:
                rules=create_rules(df, sup/20, conf/15, max_len, apriori)
                stat={
                    "count":rules.shape[0],
                    "sup":sup/20,
                    "conf":conf/15,
                    "max_len":max_len,
                    "mean_lift":rules["lift"].mean(),
                    "max_lift":rules["lift"].max()
                }
                rules_stats.append(stat)
            except:
                #иногда получаются пустые множества, которые кидают ероры при вычислении метрик
                pass

In [8]:
from operator import itemgetter
max_rules_ap=rules_stats[0]
max_mean_ap=rules_stats[0]
for stat in rules_stats:
    if stat['count']>max_rules_ap['count'] :
        max_rules_ap=stat
    if stat['mean_lift']>max_mean_ap['mean_lift'] :
        max_mean_ap=stat
print("max rules : ", max_rules_ap)
print("max mean : ", max_mean_ap)

max rules :  {'count': 1452, 'sup': 0.05, 'conf': 0.06666666666666667, 'max_len': 3, 'mean_lift': 1.0755676650037984, 'max_lift': 1.1644891366016128}
max mean :  {'count': 12, 'sup': 0.1, 'conf': 0.4666666666666667, 'max_len': 3, 'mean_lift': 1.0899102183690281, 'max_lift': 1.1644891366016128}


Як бачимо, при такій значній різниці в кількості правил середнє
покращення майже не змінилося, але їхня достовірність та підтримка стала набагато вища

Побудувати частi набори та множину АП, використовуючи алгоритм FP-
росту. Дослiдити множини АП залежно вiд параметрiв побудови АП, якi
було наведено вище для Apriori.
Пiдiбрати значення параметрiв алгоритму FP-росту, якi призводять до
значущих АП.

В основе метода лежит предобработка базы транзакций, в процессе которой эта база
данных преобразуется в компактную древовидную структуру, называемую Frequent-Pattern Tree
 – дерево популярных предметных наборов.

Побудувати частi набори та множину АП, використовуючи алгоритм FP-
росту. Дослiдити множини АП залежно вiд параметрiв побудови АП, якi
було наведено вище для Apriori.
Пiдiбрати значення параметрiв алгоритму FP-росту, якi призводять до
значущих АП.

В основе метода лежит предобработка базы транзакций, в процессе которой эта база
данных преобразуется в компактную древовидную структуру, называемую Frequent-Pattern Tree
 – дерево популярных предметных наборов.

In [9]:
rules_stats=[]
for sup in range(1,10):
    for conf in range(1,10):
        for max_len in range(2,12):
            try:
                rules=create_rules(df, sup/20, conf/15, max_len, fpgrowth)
                stat={
                    "count":rules.shape[0],
                    "sup":sup/20,
                    "conf":conf/15,
                    "max_len":max_len,
                    "mean_lift":rules["lift"].mean(),
                    "max_lift":rules["lift"].max()
                }
                rules_stats.append(stat)
            except:
                pass

In [10]:
max_rules_fp=rules_stats[0]
max_mean_fp=rules_stats[0]
for stat in rules_stats:
    if stat['count']>max_rules_fp['count'] :
        max_rules_fp=stat
    if stat['mean_lift']>max_mean_fp['mean_lift'] :
        max_mean_fp=stat
print("max rules : ", max_rules_fp)
print("max mean : ", max_mean_fp)

max rules :  {'count': 1452, 'sup': 0.05, 'conf': 0.06666666666666667, 'max_len': 3, 'mean_lift': 1.0755676650037984, 'max_lift': 1.1644891366016128}
max mean :  {'count': 12, 'sup': 0.1, 'conf': 0.4666666666666667, 'max_len': 3, 'mean_lift': 1.0899102183690281, 'max_lift': 1.1644891366016128}


Результати такі ж, як і у апріорі алгоритму

## Прогнози
(на основі меньшої множини)

In [16]:
rules = create_rules(df, max_mean_fp['sup'], max_mean_fp['conf'], max_mean_fp['max_len'], fpgrowth)
rules[["antecedents",	"consequents"]]

,antecedents,consequents
2,(Ghee),(Coffee Powder)
3,(Coffee Powder),(Ghee)
20,(Sugar),(Butter)
21,(Butter),(Sugar)
53,(Lassi),(Milk)
58,(Sweet),(Lassi)
59,(Lassi),(Sweet)
62,"(Panner, Sweet)",(Lassi)
63,"(Panner, Lassi)",(Sweet)
64,"(Sweet, Lassi)",(Panner)


Порівняємо швидкість (шоб хоч щось)

In [14]:
print("apriori")
%timeit create_rules(df, max_rules_fp['sup'], max_rules_fp['conf'], max_rules_fp['max_len'], apriori)
print("fpgrowth")
%timeit create_rules(df, max_rules_fp['sup'], max_rules_fp['conf'], max_rules_fp['max_len'], fpgrowth)

apriori
141 ms ± 6.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
fpgrowth
284 ms ± 55.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


На данному датасеті апріорі алгоритм більш бистрий